In [1]:
import pandas as pd
import numpy as np
import jieba


In [2]:
train_df = pd.read_csv("./AutoMaster_TrainSet.csv").dropna(how="all")
test_df = pd.read_csv("./AutoMaster_TestSet.csv").dropna(how="all")

FileNotFoundError: [Errno 2] File b'./AutoMaster_TrainSet.csv' does not exist: b'./AutoMaster_TrainSet.csv'

In [24]:
def build_train_test_val_list(train_df, test_df):
    num_val = int(len(train_df) * 0.1)
    num_train = len(train_df) - num_val
    num_test = len(test_df)

    def df2list(df, num):
        # train or val
        if df.shape[1] == 6:
            # train
            if num > 0:
                x_df = df.iloc[:num, 1].map(str) + "/"
                for i in range(2, 5):
                    x_df += "/" + df.iloc[:num, i].map(str)
                y_df = df.iloc[:num,5]
            else: # val
                x_df = df.iloc[num:, 1].map(str) + "/"
                for i in range(2, 5):
                    x_df += "/" + df.iloc[num:, i].map(str)
                y_df = df.iloc[num:,5]
            return x_df.tolist(), y_df.tolist()
        else: # test
            x_df = df.iloc[:num, 1].map(str) + "/"
            for i in range(2, 5):
                x_df += "/" + df.iloc[:num, i].map(str)
            return x_df.tolist()

    train_x_list, train_y_list = df2list(train_df, num_train)
    test_x_list = df2list(test_df, num_test)
    val_x_list, val_y_list = df2list(train_df, -num_train)

    return [train_x_list, train_y_list, test_x_list, val_x_list, val_y_list]
    
    
train_x_list, train_y_list, test_x_list = build_train_test_dataset(train_df,test_df)

length of train_y:  82943
train_y done
length of train_x:  82943
train_x done
length of test_x:  20000
test_x done


In [25]:
train_y_list[0]

['随时', '联系']

In [26]:
def save_train_test(train_x_list, train_y_list, test_x_list):
    with open("train_x.txt", "w") as f:
        for sentence_list in train_x_list:
            f.write(" ".join(sentence_list) + "\n")
    with open("train_y.txt", "w") as f:
        for sentence_list in train_y_list:
            f.write(" ".join(sentence_list) + "\n")  
    with open("test_x.txt", "w") as f:
        for sentence_list in test_x_list:
            f.write(" ".join(sentence_list) + "\n")
    
save_train_test(train_x_list, train_y_list, test_x_list)

In [27]:
def load_train_test(train_x, train_y, test_x):
    train_x_list, train_y_list, test_x_list = [], [], []
    with open(train_x, "r") as f:
        for line in f.readlines():
            line = [s for s in line.strip().split(" ") if s != ""]
            train_x_list.append(line)
    with open(train_y, "r") as f:
        for line in f.readlines():
            line = [s for s in line.strip().split(" ") if s != ""]
            train_y_list.append(line)
    with open(test_x, "r") as f:
        for line in f.readlines():
            line = [s for s in line.strip().split(" ") if s != ""]
            test_x_list.append(line)
    return train_x_list, train_y_list, test_x_list

train_x_list, train_y_list, test_x_list = load_train_test("train_x.txt","train_y.txt","test_x.txt")

In [37]:
from collections import Counter
class Vocab():
    def __init__(self, word_count=None):
        self.id2word = {0:"<UNK>", 1:"<START>" , 2:"<END>", 3:"<PAD>"}
        self.word2id = {"<UNK>":0, "<START>":1, "<END>":2, "<PAD>":3}
#         self.word_count = {"UNK":0}
        self.size = 5
        
    def add(self, word):
#         if word in self.word2id:
#             self.word_count[word] += 1
#         else:
        self.id2word[self.size] = word
        self.word2id[word] = self.size
#             self.word_count[word] = 1
        self.size += 1  
            
    def __len__(self):
        return self.size
    
    def save(self, path="vocab.txt"):
        with open(path , "w") as f:
            for word, idx in self.word2id.items():
                f.write(f"{word} {idx}\n")
        print(f"vocab saved in {path}, vocab size: {self.size}")
                
    def reload(self, path="vocab.txt"):
        with open(path, "r") as f:
            idx = 1
            for line in f.readlines():
                sentence = line.strip().split()
                self.id2word[idx] = word
                self.word2id[word] = idx
        self.size =len(self.id2word)
        print(f"vocab loaded from {path}")

In [38]:

def build_vocab(threshold=0.9):
    "统计词频，根据给定的阈值筛选后，保存至词典"
    all_context_list = []
    for x in [train_x_list, train_y_list, test_x_list, val_x_list, val_y_list]:
        for y in x:
            for word in y:
                all_context_list.append(word)
                
    cnt = Counter(all_context_list)
    all_words_num = len(cnt)
    words_num_after = int(threshold * all_words_num)
    print(f"all words num {all_words_num}, words after {words_num_after}, {all_words_num - words_num_after} were throwed")
    
    vocab = Vocab()
    for char, freq in cnt.most_common(words_num_after):
        if freq > 3:
            vocab.add(char)
    print(f"vocab size {len(vocab)}")
    vocab.save()
    return vocab
       
vocab = build_vocab()

vocab saved in vocab.txt, vocab size: 22063


In [1042]:
class Config():
    def __init__(self):
        self.input_size = 200
        self.hidden_size = 64
        self.dropout = 0.5
        self.num_layers = 1
        self.bidirectional = True
        self.batch_size = 1
        self.input_max_len = 256
        self.output_max_len = 64
config = Config()

In [1043]:
from torch.utils.data import Dataset
from tqdm import tqdm
class TextDataset(Dataset):
    def __init__(self, config, vocab, mode="train", num_data=None):
        self.mode = mode
        self.input_max_len = config.input_max_len
        self.output_max_len = config.output_max_len
        self.vocab = vocab
        if mode=="train":
            self.train_x = []
            self.train_y = []
#             self.size = len(self.train_x)
            
            with open("train_x.txt", "r") as f:
                for line in tqdm(f.readlines()[:num_data]):
                    line = [s for s in line.strip().split(" ") if s != ""]
                    
                    if len(line) < self.input_max_len:
                        line += ["<PAD>"]*(self.input_max_len - len(line))
                    else:
                        line = line[:self.input_max_len]
                        
                    self.train_x.append(line)
                    
            with open("train_y.txt", "r") as f:
                for line in tqdm(f.readlines()[:num_data]):
                    line = [s for s in line.strip().split(" ") if s != ""]
                    if len(line) < self.output_max_len:
                        line += ["<END>"] + ["<PAD>"] * (self.output_max_len - len(line)-1)  
                    else:
                        line = line[:self.output_max_len-1] + ["<END>"]
                        
                    self.train_y.append(line)
                    
            self.size = len(self.train_x)
        elif mode=="eval":
            self.eval_x = []
            self.eval_y = []
#             self.size = len(self.train_x)
            with open("train_x.txt", "r") as f: 
                for line in tqdm(f.readlines()[-num_data:]):
                    line = [s for s in line.strip().split(" ") if s != ""]
                    if len(line) < self.input_max_len:
                        line += ["<PAD>"]*(self.input_max_len - len(line))
                    else:
                        line = line[:self.input_max_len]
                    self.eval_x.append(line)
                    
            with open("train_y.txt", "r") as f:
                for line in tqdm(f.readlines()[-num_data:]):
                    line = [s for s in line.strip().split(" ") if s != ""]
                    self.eval_y.append(line)
            self.size = len(self.eval_x)
            
            
        else:
            self.test_x = []
            with open("test_x.txt", "r") as f:
                for line in tqdm(f.readlines()[:num_data]):
                    line = [s for s in line.strip().split(" ") if s != ""]
                    if len(line) < self.input_max_len:
                        line += ["<PAD>"]*(self.input_max_len - len(line))
                    else:
                        line = line[:self.input_max_len]
                    self.test_x.append(line)
            self.size = len(self.test_x)
            
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        if self.mode=="train":
            train_x_tokens = []
            for x in self.train_x[idx]:
                if x not in self.vocab.word2id:
                    train_x_tokens.append(self.vocab.word2id["<UNK>"])
                else:
                    train_x_tokens.append(self.vocab.word2id[x])
            
            train_y_tokens = []
            for x in self.train_y[idx]:
                if x not in self.vocab.word2id:
                    train_y_tokens.append(self.vocab.word2id["<UNK>"])
                else:
                    train_y_tokens.append(self.vocab.word2id[x])
            
            return torch.tensor(train_x_tokens), torch.tensor(train_y_tokens)
        
        elif self.mode == "eval":
            eval_x_tokens = []
            for x in self.eval_x[idx]:
                if x not in self.vocab.word2id:
                    eval_x_tokens.append(self.vocab.word2id["<UNK>"])
                else:
                    eval_x_tokens.append(self.vocab.word2id[x])
            
            return torch.tensor(eval_x_tokens), self.eval_y[idx]
        
        else:
            test_x_tokens = []
            for x in self.test_x[idx]:
                if x not in self.vocab.word2id:
                    test_x_tokens.append(self.vocab.word2id["<UNK>"])
                else:
                    test_x_tokens.append(self.vocab.word2id[x])
            return torch.tensor(test_x_tokens)
    

In [1044]:
train_dataset = TextDataset(config,vocab, "train",1000)
print(len(train_dataset))
eval_dataset = TextDataset(config,vocab, "eval",100)
print(len(eval_dataset))
test_dataset = TextDataset(config,vocab, "test", 500)



100%|██████████| 30000/30000 [00:00<00:00, 163094.82it/s]


30000


100%|██████████| 1000/1000 [00:00<00:00, 50445.66it/s]

100


In [1045]:
# print("".join([vocab.id2word[x.item()] for x in sample[0][0]]))
# print(sample[0][1])
# # print("".join([vocab.id2word[x.item()] for x in sample[1]]))
# print("".join([vocab.id2word[x.item()] for x in sample_t]))

In [1046]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True) 
eval_dataloader = DataLoader(eval_dataset, batch_size=1, shuffle=False)

In [1047]:
train_x, train_y = next(iter(train_dataloader)) # seq_len, batch_size * 1
test_x = next(iter(test_dataloader)) # seq_len, batch_size * 1
val_x, val_y = next(iter(eval_dataloader))
print(train_x.shape, train_y.shape, test_x.shape)

torch.Size([64, 50]) torch.Size([64, 20]) torch.Size([1, 50])


In [1048]:
class Encoder(torch.nn.Module):
    def __init__(self, config, vocab):
        super(Encoder, self).__init__()
        self.vocab = vocab
        self.input_size = config.input_size
        self.hidden_size = config.hidden_size
        self.num_layers = config.num_layers
        self.bidirectional = config.bidirectional
        self.batch_size = config.batch_size
        
        self.embedding = torch.nn.Embedding(len(self.vocab), self.input_size)
        
        self.lstm = torch.nn.LSTM(self.input_size, self.hidden_size, num_layers=self.num_layers
                                  , batch_first=True, bidirectional=self.bidirectional)
        
        self.dropout = torch.nn.Dropout(config.dropout)
    
    def forward(self, inputs, hidden=None, cell=None):
        '''
        inputs: batch_size, seq_len
        init_hidden/init_cell: numb_layers * 2, batch_size, hidden_size
        ''' 
#         print("encoder input (batch_size, seq_len): ", inputs.shape) 
        inputs = self.embedding(inputs)
#         print("encoder input after embed (batch_size, seq_len, input_size):", inputs.shape)
        # inputs after embedding: batch_size, seq_len, input_size
        if hidden is not None:
            outputs, (hidden, cell) = self.lstm(inputs, (hidden, cell))
#             print("final output shape: ", outputs.shape)
#             print("final hidden shape: ", hidden.shape)
#             print("final cell shape: ", cell.shape)
        else:
            outputs, (hidden, cell) = self.lstm(inputs)
#             print("final output shape: ", outputs.shape)
#             print("final hidden shape: ", hidden.shape)
#             print("final cell shape: ", cell.shape)
        

        return outputs, hidden, cell
    
    def init_hidden_cell(self):
        D = self.num_layers * 2 if self.bidirectional else self.num_layers
        return torch.zeros(D, self.batch_size, self.hidden_size),torch.zeros(D, self.batch_size, self.hidden_size)

In [1049]:
encoder = Encoder(config,vocab)

In [1050]:
init_hidden, init_cell = encoder.init_hidden_cell()
out, hidden, cell = encoder(train_x, init_hidden, init_cell)
out.shape, hidden.shape, cell.shape

(torch.Size([64, 50, 24]), torch.Size([2, 64, 12]), torch.Size([2, 64, 12]))

In [1051]:
class Decoder(torch.nn.Module):
    def __init__(self, config, vocab):
        super(Decoder, self).__init__()
        self.vocab = vocab
        self.input_size = config.input_size
        self.output_size = len(vocab)
        self.hidden_size = config.hidden_size
        self.dropout = torch.nn.Dropout(config.dropout)
        self.num_layers = config.num_layers
        self.bidirectional = config.bidirectional
        
        self.embedding = torch.nn.Embedding(len(self.vocab), self.input_size)
        self.lstm = torch.nn.LSTM(self.input_size, self.hidden_size, num_layers=self.num_layers
                                  , batch_first=True, bidirectional=self.bidirectional)
        self.fc = torch.nn.Linear(self.hidden_size * 2, self.output_size)
        self.softmax = torch.nn.LogSoftmax(dim=1)
    
    def forward(self, inputs, hidden, cell):
        '''
        inputs: batch_size, seq_len
        init_hidden/init_cell: numb_layers * 2, batch_size, input_size
        '''
#         print("decoder input:", inputs.shape)
        inputs = self.embedding(inputs) 
#         print("decoder input after embed:", inputs.shape)
#         inputs after embedding: batch_size, seq_len, input_size
#         print("decoder input after embed and permute (batch_size,seq_len,input_size):", inputs.shape)
#         print("hidden: ", hidden.shape)
        outputs, (hidden, cell) = self.lstm(inputs, (hidden, cell))
#         print("outputs shape: ", outputs.shape)
        prediction = self.softmax(self.fc(torch.squeeze(outputs, 1)))
#         print(f"prediction: {prediction.shape}    outputs: {outputs.shape}")
        return prediction, outputs, hidden, cell
    


In [1052]:
decoder = Decoder(config, vocab)

In [1053]:
decoder_input = torch.tensor([vocab.word2id["<START>"]] * config.batch_size).view(-1,1)
# decoder_input = torch.unsqueeze(decoder_input, 1)
decoder_input.shape

torch.Size([64, 1])

In [1054]:
prediction, outputs, next_hidden, next_cell = decoder(decoder_input, hidden, cell)
print("predictions----", prediction.shape)
print("outputs----", outputs.shape)
# print("next_hidden----", next_hidden.shape)
predict_token = torch.argmax(prediction,1)
# predict_token, vocab.id2word[predict_token.item()]

print("pred_token:", predict_token)
print(f"next hidden: {next_hidden.shape}")

predictions---- torch.Size([64, 22063])
outputs---- torch.Size([64, 1, 24])
pred_token: tensor([ 9331, 20323, 21070, 21070, 15452,  1996, 21414, 21070, 20442, 21414,
        20323, 20442, 20442, 21070,  1996, 20106, 20106, 20106,  5269, 21070,
        16666,  9811,  1869,  2731, 21070, 20442,  7465, 21005, 21070, 21070,
         9811, 21070, 21070,  9160, 11217,  1996, 21244, 21070,  4879, 21070,
        16883,  1264, 21070, 21070,  3124, 20442, 21070, 20442, 21711, 10742,
        21070, 21070, 20323,  9811,  1996, 21070, 21070, 20106, 21070,  8152,
        21414,  1996, 21414, 20913])
next hidden: torch.Size([2, 64, 12])


In [1055]:
# decoder_input = torch.tensor([predict_token])
decoder_input = predict_token.view(-1,1)
decoder_input.shape

torch.Size([64, 1])

In [1056]:
final_prediction, final_outputs, final_hidden, final_cell = decoder(decoder_input, next_hidden, next_cell)
final_predict_token = torch.argmax(final_prediction,1)
final_predict_token.shape

torch.Size([64])

In [1057]:
class Seq2Seq(torch.nn.Module):
    def __init__(self, config, vocab):
        super(Seq2Seq, self).__init__()
        self.config = config
        self.input_max_len = config.input_max_len
        self.output_max_len = config.output_max_len
        self.encoder = Encoder(config,vocab)
        self.decoder = Decoder(config,vocab)
        
    def forward(self, encoder_input, target=None):
        target_len = self.output_max_len
        batch_size = encoder_input.shape[0]
#         if target is not None:
#             batch_size = self.config.batch_size
#         else:
#             batch_size = 1
        
        outputs = torch.zeros(batch_size, target_len, len(vocab))
        
        
        decoder_input = torch.tensor([vocab.word2id["<START>"]] * batch_size).view(-1,1)
        
        encoder_outputs, hiddens, cells = encoder(encoder_input)
#         predictions = []
#         print("-------encoder-------")
#         print("encoder outputs: ", encoder_outputs.shape)
#         print("encoder hiddens: ", hiddens.shape)
#         print("encoder cells: ", cells.shape)
#         print("-------decoder-------")
        for i in range(target_len):
#             print(f"---------step {i}----------")
#             print("decoder outputs: ", encoder_outputs.shape)
#             print("decoder hiddens: ", hiddens.shape)
#             print("decoder cells: ", cells.shape)
            next_predictions, next_outputs, hiddens, cells = decoder(decoder_input, hiddens, cells)
            outputs[:,i,:] = next_predictions
#             predictions[:,i] = 
#             predictions_tokens = next_predictions.argmax(1)
#             predictions.append(predictions_tokens)
            
#             print([vocab.id2word[x] for x in predictions_tokens])
        return outputs     

In [1058]:
model = Seq2Seq(config,vocab)

In [1059]:
outputs = model(train_x, train_y)

In [1060]:
outputs.shape

torch.Size([64, 20, 22063])

In [1061]:
outputs.shape, train_y.shape

(torch.Size([64, 20, 22063]), torch.Size([64, 20]))

In [1062]:
model_PATH  = 'best.pkl'
predictions = outputs.argmax(2)
predictions.shape

torch.Size([64, 20])

In [1]:
def loss_fn(outputs, train_y):
    '''
    outputs: batch_size, seq_len, input_size
    train_y: batch_size, seq_len
    '''
    loss_obj = torch.nn.NLLLoss(reduction='mean')
    batch_loss = 0
    batch_size = train_y.shape[1]
    for i in range(batch_size):
        batch_outputs = outputs[:,i,:]
        batch_y = train_y[:,i]
        batch_loss += loss_obj(batch_outputs, batch_y)
    return batch_loss / batch_size

loss_fn(outputs,train_y)

NameError: name 'outputs' is not defined

In [1064]:
def lcs(s1, s2):
    matrix = [ [0 for x in range(len(s2))] for x in range(len(s1)) ]
#     cs = ""
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i]==s2[j]:
                if i==0 or j==0:
                    matrix[i][j] = 1
#                     cs += s1[i]
                else:
                    matrix[i][j] = matrix[i-1][j-1] + 1
#                     cs += s1[i]
            else:
                if i==0 or j==0:
                    matrix[i][j] = 0
                else:
                    matrix[i][j] = max(matrix[i-1][j], matrix[i][j-1])

    return matrix[len(s1)-1][len(s2)-1]

lcs(["a", "b","c", "e", "f"], ["c", "e", "f"])

3

In [1065]:
def rouge(pred, target):
    beta = 100
    temp_lcs = lcs(pred, target)
    r_lcs = temp_lcs / len(target)
    p_lcs = temp_lcs / len(pred)
    up = (1 + beta**2) * r_lcs * p_lcs
    down = r_lcs + beta**2 * p_lcs + 0.001
    return up/down

def evaluate(model, eval_dataloader):
#     print(eval_dataset[0][1])
    model.eval()
    total_rouge = 0
    with torch.no_grad():
        for dev_x, target in eval_dataloader:
            target = [vocab.id2word[x.item()] for x in target[0]]
            outputs = model(dev_x)
            predictions = outputs.argmax(2)[0] # predictions: 64,50
#             print(f"----------predictions {predictions.shape}--------------")
            result = []
            for one_pred in predictions:
#                 one_line_tokens = one_pred
                pred = vocab.id2word[one_pred.item()]
                if pred == "<END>": break
                result.append(pred)
            
            total_rouge += rouge(result, target)
    return total_rouge
         
evaluate(model, eval_dataloader)

0.0

In [1066]:
def train(model, train_dataloader, eval_dataloader, epoch_num):
#     total_loss = 0
    loss_ = loss_fn
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
    for epoch in range(epoch_num):
        model.train()
        total_loss = 0
        best_test_loss = 1000
        print(f"-----------------epoch {epoch}-------------------")
        for step, (train_x, train_y) in enumerate(train_dataloader):
            outputs = model(train_x, train_y)
#             print(outputs.shape, train_y.shape)
            loss = loss_(outputs, train_y)
            # 单向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss
            if step // 10 == 0: 
                print(f"step {step}  loss {loss.item()} average_loss {total_loss / (step + 1)}")
                predictions = outputs.argmax(2)[0]
                result = []
                target_sentence = []
                for tr in train_y[0]:
                    tr = vocab.id2word[tr.item()]
                    target_sentence.append(tr)
                for one_pred in predictions:
    #                 one_line_tokens = one_pred
                    pred = vocab.id2word[one_pred.item()]
                    if pred == "<END>": break
                    result.append(pred)
                
                print(f"sample result: {result[:10]} \ntarget sentence: {target_sentence}")
        print(f"batch ROUGE-L: {evaluate(model, eval_dataloader)}")
        

In [1067]:
train(model, train_dataloader, eval_dataloader, 5)

-----------------epoch 0-------------------
step 0  loss 9.964241981506348 average_loss 9.964241981506348
sample result: ['手指', '型号', '型号', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌']
step 1  loss 9.995153427124023 average_loss 9.979698181152344
sample result: ['故障诊断', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌']
step 2  loss 9.979910850524902 average_loss 9.979769706726074
sample result: ['886', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌']
step 3  loss 9.991281509399414 average_loss 9.982646942138672
sample result: ['型号', '型号', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌']
step 4  loss 9.998929023742676 average_loss 9.985902786254883
sample result: ['3s', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌']
step 5  loss 10.00325870513916 average_loss 9.988795280456543
sample result: ['手指', '886', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌']
step 6  loss 9.98540210723877 average_loss 9.988310813903809
sample result: ['丝扣', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌', '锁舌',